In [ ]:
# @title 1. Install Library & Download Dataset YOLO
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="TjWK841PaXoG3k7fu8P7")
project = rf.workspace("ranias-workspace").project("dataset-plat-nomor")
version = project.version(5)
dataset = version.download("yolov8")



print(f"\n✅ Dataset terdownload di folder: {dataset.location}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 130.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Dataset-plat-nomor-5 in yolov8:: 100%|██████████| 3760/3760 [00:00<00:00, 6192.91it/s]


✅ Dataset terdownload di folder: /content/Dataset-plat-nomor-5


In [ ]:
# @title 2. Training YOLOv8
import os
!pip install ultralytics
from ultralytics import YOLO

# Cek apakah dataset sudah terdownload
if not 'dataset' in locals() and not os.path.exists("dataset-plat-nomor-5"):
    print("⚠️ Dataset belum terdeteksi. Pastikan Langkah 1 sudah dijalankan!")
else:
    print("✅ Library YOLO siap. Memulai konfigurasi...")
    model_yolo = YOLO('yolov8n.pt')

    # Cari file data.yaml otomatis
    data_yaml_path = f"{dataset.location}/data.yaml"

    print(f"📂 Menggunakan konfigurasi data dari: {data_yaml_path}")

    results = model_yolo.train(data=data_yaml_path, epochs=50, imgsz=640)

    print("✅ Training Selesai! Model siap digunakan.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Library YOLO siap. Memulai konfigurasi...
📂 Menggunakan konfigurasi data dari: /content/Dataset-plat-nomor-5/data.yaml
Ultralytics 8.3.243 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Dataset-plat-nomor-5/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, 

In [ ]:
# @title 3. Siapkan Dataset Huruf
import os
import shutil
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')
PATH_FOLDER_DRIVE = '/content/drive/MyDrive/Dataset_Plat'
DATASET_KNN_PATH = '/content/dataset_huruf'

if not os.path.exists(DATASET_KNN_PATH):
    print(f"🚀 Sedang menyalin data dari Drive: {PATH_FOLDER_DRIVE}")
    print("⏳ Mohon bersabar, menyalin ribuan file folder biasa agak lama...")

    try:
        # Menyalin seluruh isi folder
        shutil.copytree(PATH_FOLDER_DRIVE, DATASET_KNN_PATH)
        print(f"✅ SUKSES! Dataset tersalin ke: {DATASET_KNN_PATH}")
    except FileNotFoundError:
        print("❌ ERROR: Folder tidak ditemukan! Cek lagi nama folder di variabel PATH_FOLDER_DRIVE.")
    except Exception as e:
        print(f"❌ Error lain: {e}")
else:
    print("✅ Dataset sudah siap di Local Colab.")

Mounted at /content/drive
🚀 Sedang menyalin data dari Drive: /content/drive/MyDrive/Dataset_Plat
⏳ Mohon bersabar, menyalin ribuan file folder biasa agak lama...
✅ SUKSES! Dataset tersalin ke: /content/dataset_huruf


In [ ]:
# @title 4. Fungsi Deteksi & Segmentasi
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load Model YOLO
best_model_path = '/content/runs/detect/train/weights/best.pt'
if os.path.exists(best_model_path):
    yolo_plate_model = YOLO(best_model_path)
else:
    print("⚠️ Model YOLO tidak ditemukan! Pastikan sudah training (Cell 2) atau Load dari Drive.")

def preprocess_image(image):
    """Hanya untuk mencari kontur/lokasi huruf (Bukan untuk diambil gambarnya)"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 19, 5)

    kernel = np.ones((3,3), np.uint8)
    # Closing untuk menyatukan bagian huruf yang terpisah
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    return binary

def normalize_character(char_img, size=(20, 30)):
    return cv2.resize(char_img, size)

def locate_plate_yolo(image):
    results = yolo_plate_model(image, verbose=False)
    for result in results:
        boxes = result.boxes
        if len(boxes) > 0:
            box = boxes[0]
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cropped = image[y1:y2, x1:x2]
            return cropped
    return None

def segment_characters(binary_img, plate_img_gray):
    """
    Update: Memotong huruf dari gambar GRAYSCALE (Plate Asli),
    bukan dari binary yang kotor.
    """
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    chars = []

    # Urutkan dari kiri ke kanan
    boundingBoxes = [cv2.boundingRect(c) for c in contours]
    if len(contours) > 0:
        (contours, boundingBoxes) = zip(*sorted(zip(contours, boundingBoxes), key=lambda b: b[1][0]))

    h_plat, w_plat = binary_img.shape

    potential_chars = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h
        area = w * h

        # Filter Ukuran
        if h > h_plat * 0.35 and h < h_plat * 0.95 and area > 100 and 0.15 < aspect_ratio < 1.0:
             potential_chars.append((x, y, w, h))

    # Y-Alignment & Crop Raw
    if len(potential_chars) > 0:
        y_positions = [item[1] for item in potential_chars]
        median_y = np.median(y_positions)
        y_tolerance = h_plat * 0.2

        # Urut X axis
        potential_chars = sorted(potential_chars, key=lambda item: item[0])

        for (x, y, w, h) in potential_chars:
            if abs(y - median_y) < y_tolerance:
                roi_gray = plate_img_gray[y:y+h, x:x+w]
                roi_gray = cv2.copyMakeBorder(roi_gray, 5, 5, 5, 5, cv2.BORDER_CONSTANT, value=0)
                chars.append(roi_gray)

    return chars

In [ ]:
# @title 5: Setup & Training Otak
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
from ultralytics import YOLO
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
# --- 1. SETUP YOLO (PENCARI PLAT) ---
# Load model YOLO yang sudah ada (dari training sebelumnya)
best_model_path = '/content/runs/detect/train/weights/best.pt'
if os.path.exists(best_model_path):
    yolo_plate_model = YOLO(best_model_path)
    print("✅ Model YOLO (Pencari Plat) Berhasil Dimuat!")
else:
    print("⚠️ Model YOLO belum training! Gunakan yolov8n.pt standar (kurang akurat).")
    yolo_plate_model = YOLO('yolov8n.pt')

def locate_plate_yolo(image):
    results = yolo_plate_model(image, verbose=False)
    for result in results:
        boxes = result.boxes
        if len(boxes) > 0:
            box = boxes[0]
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cropped = image[y1:y2, x1:x2]
            return cropped
    return None

def preprocess_image(image):
    if len(image.shape) == 3: gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else: gray = image
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 19, 5)
    kernel = np.ones((3,3), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    return binary

# --- 2. FUNGSI HOG HD (MATA DEWA) ---
def extract_hog_features_hd(image):
    # Resize HD (40x64)
    target_size = (40, 64)
    resized = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

    # Adaptive Threshold (Menjaga topi angka 7)
    binary = cv2.adaptiveThreshold(resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 13, 5)

    # Dilation Tipis (Menyambung garis putus)
    kernel = np.ones((2,2), np.uint8)
    binary = cv2.dilate(binary, kernel, iterations=1)

    # Ekstrak HOG
    fd, _ = hog(binary, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    return fd, binary

# --- 3. FUNGSI POTONG HURUF ---
def segment_characters_hd(binary_img, plate_img_gray):
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    chars = []
    h_plat, w_plat = binary_img.shape
    potential_chars = []

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect = w / h
        area = w * h
        if h > h_plat*0.35 and h < h_plat*0.95 and area > 100 and 0.15 < aspect < 1.0:
            potential_chars.append((x, y, w, h))

    if potential_chars:
        potential_chars.sort(key=lambda x: x[0])
        ys = [c[1] for c in potential_chars]
        median_y = np.median(ys)
        for (x, y, w, h) in potential_chars:
            if abs(y - median_y) < h_plat * 0.2:
                roi_gray = plate_img_gray[y:y+h, x:x+w]
                # Padding 3px
                roi_gray = cv2.copyMakeBorder(roi_gray, 3, 3, 3, 3, cv2.BORDER_CONSTANT, value=0)
                chars.append(roi_gray)
    return chars

# --- 4. TRAINING KNN (PROSES LAMA DI SINI) ---
def train_knn_hd(path):
    data = []
    labels = []
    if not os.path.exists(path):
        print("❌ Dataset huruf tidak ditemukan! Cek folder dataset.")
        return None

    print("🚀 Sedang Training HOG Resolusi Tinggi (Tunggu sebentar)...")
    for folder in sorted(os.listdir(path)):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path)[:300]:
                try:
                    img = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        features, _ = extract_hog_features_hd(img)
                        data.append(features)
                        labels.append(folder)
                except: continue

    knn = KNeighborsClassifier(n_neighbors=3, metric='manhattan')
    knn.fit(data, labels)
    print("✅ TRAIN SELESAI! Model KNN siap digunakan di Cell bawah.")
    return knn

# Eksekusi Training
DATASET_KNN_PATH = '/content/dataset_huruf'
knn_final_hd = train_knn_hd(DATASET_KNN_PATH)

✅ Model YOLO (Pencari Plat) Berhasil Dimuat!
🚀 Sedang Training HOG Resolusi Tinggi (Tunggu sebentar)...
✅ TRAIN SELESAI! Model KNN siap digunakan di Cell bawah.


In [ ]:
# @title 6: Testing Foto
from google.colab import files

# Cek apakah model sudah siap
if 'knn_final_hd' not in globals() or knn_final_hd is None:
    print("⚠️ HARAP RUN CELL 1 DULU SAMPAI SELESAI!")
else:
    uploaded = files.upload()

    for fn in uploaded.keys():
        print(f"\n🚘 SEDANG MEMPROSES: {fn}")
        img = cv2.imread(fn)

        # 1. YOLO (Cari Plat)
        plate_img = locate_plate_yolo(img)
        if plate_img is None:
            print("❌ Plat tidak ditemukan YOLO.")
            continue

        # 2. Potong Huruf
        if len(plate_img.shape) == 3: plate_gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
        else: plate_gray = plate_img

        guide = preprocess_image(plate_img)
        chars = segment_characters_hd(guide, plate_gray)

        # Tampilkan Plat
        plt.figure(figsize=(5,2))
        plt.imshow(cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB))
        plt.axis('off'); plt.show()

        # 3. Baca Huruf pakai Model dari Cell 1
        res_text = ""
        if chars:
            fig, axes = plt.subplots(1, len(chars), figsize=(12, 2))
            if len(chars)==1: axes=[axes]

            for i, char in enumerate(chars):
                # Ekstrak fitur input
                features_input, bin_view = extract_hog_features_hd(char)

                # Prediksi
                pred = knn_final_hd.predict([features_input])[0]
                res_text += pred

                # Tampilkan
                axes[i].imshow(bin_view, cmap='gray')
                axes[i].set_title(pred, color='green', fontweight='bold', fontsize=16)
                axes[i].axis('off')
            plt.show()
            print(f"📝 HASIL BACAAN: {res_text}")
        else:
            print("❌ Huruf tidak terdeteksi.")

In [ ]:
# @title Batch Processing + F1-Score
import os
import cv2
import pandas as pd
from IPython.display import display, HTML

# --- KONFIGURASI FOLDER ---
FOLDER_TEST = '/content/foto_test'

# 1. Cek dulu apakah Cell 1 udah dijalankan?
# Kita cek apakah fungsi 'locate_plate_yolo' dan model 'knn_final_hd' sudah ada di memori
if 'locate_plate_yolo' not in globals() or 'knn_final_hd' not in globals():
    print("⚠️ ERROR: Harap RUN CELL 1 dulu sampai selesai!")
    print("👉 Kita butuh fungsi 'locate_plate_yolo' dan model 'knn_final_hd' dari sana.")
else:
    # 2. Cek Folder Foto
    if not os.path.exists(FOLDER_TEST):
        os.makedirs(FOLDER_TEST)
        print(f"📂 Folder '{FOLDER_TEST}' dibuat. Masukkan foto dulu!")
    else:
        files_test = sorted([f for f in os.listdir(FOLDER_TEST) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])

        if len(files_test) == 0:
            print(f"⚠️ Folder '{FOLDER_TEST}' KOSONG! Upload foto dulu.")
        else:
            print(f"🚀 Memproses {len(files_test)} foto... (Menghitung F1-Score)")

            rekap_hasil = []

            # --- VARIABEL HITUNG SCORE ---
            tp = 0 # Benar (True Positive)
            fp = 0 # Salah Baca (False Positive)
            fn = 0 # Gagal Deteksi (False Negative)

            for fn_name in files_test:
                path_img = os.path.join(FOLDER_TEST, fn_name)
                # Kunci Jawaban (Nama File)
                nama_asli = os.path.splitext(fn_name)[0].replace(" ", "").upper()

                img = cv2.imread(path_img)
                if img is None: continue

                # --- 1. DETEKSI PLAT (Pake Fungsi Lama yg Terbukti Jalan) ---
                plate_img = locate_plate_yolo(img)

                pred_text = ""
                status_detail = ""

                if plate_img is None:
                    # Gagal Deteksi Plat
                    fn += 1
                    status_detail = "❌ FN (Gagal Deteksi)"
                    pred_text = "-"
                else:
                    # --- 2. BACA HURUF (KNN) ---
                    # Proses Preprocessing & Segmentasi
                    if len(plate_img.shape) == 3: plate_gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
                    else: plate_gray = plate_img

                    guide = preprocess_image(plate_img)
                    chars = segment_characters_hd(guide, plate_gray)

                    if chars:
                        for char in chars:
                            features_input, _ = extract_hog_features_hd(char)
                            pred = knn_final_hd.predict([features_input])[0]
                            pred_text += pred
                    else:
                        pred_text = "GAGALCROP"

                    # --- 3. CEK BENAR/SALAH ---
                    clean_pred = pred_text.replace(" ", "").upper()

                    if clean_pred == "GAGALCROP" or clean_pred == "":
                        fn += 1 # Anggap False Negative kalau gagal crop huruf
                        status_detail = "FN (Gagal Crop)"
                    elif clean_pred == nama_asli:
                        tp += 1 # Benar Sempurna
                        status_detail = "TP (Benar)"
                    else:
                        fp += 1 # Salah Baca
                        status_detail = "FP (Salah Baca)"

                # Masukkan ke list
                rekap_hasil.append({
                    "Nama File": fn_name,
                    "Kunci Jawaban": nama_asli,
                    "Prediksi AI": pred_text,
                    "Status": status_detail
                })

            # --- TAMPILKAN TABEL ---
            if rekap_hasil:
                df = pd.DataFrame(rekap_hasil)
                print("\n" + "="*50)
                print("📊 TABEL HASIL DETEKSI")
                print("="*50)
                display(df)

                # --- HITUNG F1-SCORE ---
                total_data = tp + fp + fn

                # Rumus (Pake if biar gak error bagi nol)
                presisi = (tp / (tp + fp)) * 100 if (tp + fp) > 0 else 0
                recall  = (tp / (tp + fn)) * 100 if (tp + fn) > 0 else 0
                akurasi = (tp / total_data) * 100 if total_data > 0 else 0

                if (presisi + recall) > 0:
                    f1_score = 2 * (presisi * recall) / (presisi + recall)
                else:
                    f1_score = 0

                print("\n" + "="*50)
                print("📈 LAPORAN EVALUASI MODEL (F1-SCORE)")
                print("="*50)
                print(f"Total Data      : {total_data}")
                print(f"✅ True Positive : {tp} (Benar)")
                print(f"⚠️ False Positive: {fp} (Salah Baca)")
                print(f"❌ False Negative: {fn} (Gagal Deteksi)")
                print("-" * 30)
                print(f"🎯 AKURASI   : {akurasi:.2f}%")
                print(f"🎯 PRESISI   : {presisi:.2f}%")
                print(f"🎯 RECALL    : {recall:.2f}%")
                print(f"🏆 F1-SCORE  : {f1_score:.2f}%")
                print("="*50)

🚀 Memproses 50 foto... (Menghitung F1-Score)

📊 TABEL HASIL DETEKSI


,Nama File,Kunci Jawaban,Prediksi AI,Status
0,AA4795BE.jpg,AA4795BE,GAGALCROP,FN (Gagal Crop)
1,AB4622FO.jpg,AB4622FO,OZNB42F9,FP (Salah Baca)
2,AB4848DH.jpg,AB4848DH,AB4848DH,TP (Benar)
3,AB6628YJ.jpg,AB6628YJ,A86628Y0,FP (Salah Baca)
4,AD2801BV.jpg,AD2801BV,AD2801BV,TP (Benar)
5,BH2618MA.jpg,BH2618MA,GAGALCROP,FN (Gagal Crop)
6,H2148BL.jpg,H2148BL,GAGALCROP,FN (Gagal Crop)
7,H2462FK.jpg,H2462FK,H2462FK,TP (Benar)
8,H2474VB.jpg,H2474VB,H2474VB,TP (Benar)
9,H2575AK.jpg,H2575AK,H2575AK,TP (Benar)



📈 LAPORAN EVALUASI MODEL (F1-SCORE)
Total Data      : 50
✅ True Positive : 37 (Benar)
⚠️ False Positive: 7 (Salah Baca)
❌ False Negative: 6 (Gagal Deteksi)
------------------------------
🎯 AKURASI   : 74.00%
🎯 PRESISI   : 84.09%
🎯 RECALL    : 86.05%
🏆 F1-SCORE  : 85.06%
